In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import shfl
import torch
import copy
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0-Segmentadas", 
        "csv_path": "../partitions/partition_iid_1nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 100,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder():
    return ClassifierModel(batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels

    t_test_data, t_test_label = cit_federated_government.global_model.transform_data(test_data, test_label, lb1, lb2)

    return t_federated_data, t_test_data, t_test_label

In [4]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

data, label, train_data, train_label, test_data, test_label, train_files, test_files = get_data_csv(args["data_path"], args["csv_path"], lb1)

print(len(train_data))
print(len(test_data))

681
171


In [ ]:
cit_model = cit_builder()
cit_model.train(train_data, train_label)

#t_train_data, t_train_label = cit_model.transform_data(train_data, train_label, lb1, lb2)
#t_test_data, t_test_label = cit_model.transform_data(test_data, test_label, lb1, lb2)

  0%|          | 0/77 [00:00<?, ?it/s]

[INFO] weights = [1.         0.97391304]
LAMBDA: 0.05


[1/100] Loss_D: 0.3582 Acc_D: 0.6348 Loss_G_class1: 0.0610 Loss_G_class2: 0.0593: 100%|██████████| 77/77 [01:47<00:00,  1.40s/it]
  0%|          | 0/77 [00:00<?, ?it/s]


Valid Acc = 0.6811594202898551
Valid Loss = 0.8394831019467202


[2/100] Loss_D: 0.2946 Acc_D: 0.7059 Loss_G_class1: 0.0182 Loss_G_class2: 0.0185: 100%|██████████| 77/77 [01:47<00:00,  1.40s/it]
  0%|          | 0/77 [00:00<?, ?it/s]

...

[8/100] Loss_D: 0.2226 Acc_D: 0.7966 Loss_G_class1: 0.0122 Loss_G_class2: 0.0135: 100%|██████████| 77/77 [01:48<00:00,  1.40s/it]
  0%|          | 0/77 [00:00<?, ?it/s]


Valid Acc = 0.7536231884057971
Valid Loss = 0.47884143679065333
EarlyStopping counter: 2 out of 10


[9/100] Loss_D: 0.2108 Acc_D: 0.8031 Loss_G_class1: 0.0119 Loss_G_class2: 0.0120: 100%|██████████| 77/77 [01:47<00:00,  1.40s/it]
  0%|          | 0/77 [00:00<?, ?it/s]


Valid Acc = 0.6956521739130435
Valid Loss = 0.5718406884917531
EarlyStopping counter: 3 out of 10


[10/100] Loss_D: 0.1749 Acc_D: 0.8542 Loss_G_class1: 0.0088 Loss_G_class2: 0.0118:  43%|████▎     | 33/77 [00:46<01:02,  1.41s/it]

from IPython.display import display
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import cv2


def sample_loader(sample):
    s = ToTensor()(x).float()
    s = Variable(s, requires_grad=False)
    s = s.unsqueeze(0)  
    return s

self = cit_model

for class_name in self._class_names:
    self._G_dict[class_name]= self._G_dict[class_name].to(self._device)


sample = test_data[40]
label = lb1.inverse_transform(test_label[0])[0]
x = ToTensor()(sample).float().unsqueeze(0).to(device)
class_name = 'P'
y = self._G_dict[class_name](x)
y = y[0].cpu().detach().numpy()
print(y.shape)
y = np.moveaxis(y, 0, -1)
#y = cv2.resize(y, dsize=(224, 224))
plt.imshow(y)
plt.imshow(y)


In [ ]:
metrics = cit_model.evaluate(test_data, test_label)
print("CIT Classifier Results:")
print("Loss: {}".format(metrics[0]))
print("Acc: {}".format(metrics[1]))
print("F1: {}".format(metrics[2]))
print("Precision: {}".format(metrics[3]))
print("Recall: {}".format(metrics[4]))

In [ ]:
t_train_data, t_train_label = cit_model.transform_data(train_data, train_label, lb1, lb2)
t_test_data, t_test_label = cit_model.transform_data(test_data, test_label, lb1, lb2)

from ClassifierModel import ClassifierModel

classifier_model = classifier_builder()
classifier_model.train(t_train_data, t_train_label)

In [ ]:
dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 
G_dict = cit_model._G_dict

for key, _ in G_dict.items():
    G_dict[key].to("cpu")

classifier_model.get_classification_report(test_files, dict_labels, G_dict)